This notebook demonstrates a complete trading system with:
 - Multi-asset market data loading
 - Order management system (OMS)
 - Limit order book matching engine
 - Position tracking and P&L calculation
 - Three trading strategies: trend-following, mean reversion, and arbitrage
 
 ## System Architecture
 
 ```
 MarketDataLoader → Strategy → OMS → OrderBook → PositionTracker
                      ↓              ↓           ↓
                   Signals      ExecutionReports  P&L
 ```

In [ ]:
# %%
# Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Our trading system components
from market_data_loader import MarketDataLoader
from order import Order
from oms import OrderManagementSystem
from order_book import LimitOrderBook
from position_tracker import PositionTracker

# Trading strategies
from strategies.trend_following import run_backtest as tf_backtest
from strategies.mean_reversion import run_backtest as mr_backtest
from strategies.arbitrage import run_backtest as arb_backtest

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Fetch data for multiple assets
symbols = ["AAPL", "MSFT", "GOOGL", "SPY", "QQQ"]
data = {}

print("📊 Fetching market data...")
for symbol in symbols:
    try:
        hist = loader.get_history(symbol)
        data[symbol] = hist
        print(f"✅ {symbol}: {len(hist)} days, ${hist['last_price'].iloc[-1]:.2f}")
    except Exception as e:
        print(f"❌ {symbol}: {e}")

# Show sample data
if "AAPL" in data:
    print("\n📈 Sample AAPL data:")
    print(data["AAPL"].head())

In [ ]:
# Demonstrate core trading infrastructure
print("🔧 Testing Trading Infrastructure...")

# Initialize components
oms = OrderManagementSystem()
book = LimitOrderBook("DEMO")
tracker = PositionTracker(starting_cash=100000)

# Create sample orders
orders = [
    Order("BUY1", "DEMO", "buy", 100, "limit", 150.0),
    Order("SELL1", "DEMO", "sell", 50, "limit", 149.0),  # Should match
    Order("BUY2", "DEMO", "buy", 200, "market")  # Market order
]

print("\n📋 Order Management System Test:")
for order in orders:
    try:
        # Submit to OMS
        ack = oms.new_order(order)
        print(f"✅ {order}: {ack['status']}")
        
        # Route to order book
        reports = book.add_order(order)
        print(f"   💱 Generated {len(reports)} execution reports")
        
        # Update position tracker
        for report in reports:
            tracker.update(report)
            
    except Exception as e:
        print(f"❌ {order}: {e}")

# Show final state
print(f"\n💰 Final Portfolio:")
print(f"   Cash: ${tracker.cash:,.2f}")
print(f"   Positions: {tracker.positions}")
print(f"   Book State: {book}")